In [ ]:
!pip install pandas
!pip install numpy
!pip install matplotlib
!pip install sklearn
!pip install seaborn

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
train_set = pd.read_csv("train.csv")
test_set = pd.read_csv("test.csv")
pokemons = pd.read_csv("pokemon.csv")
train_set["train"] = 1
test_set["train"] = 0
all_data = pd.concat([train_set, test_set], ignore_index=True)

In [ ]:
all_data.info()

In [ ]:
pokemons.describe(include="object")

Droppo le colonne dei nomi, in quanto valori univoci non utili

In [6]:
pokemons.drop(labels=["Name"], axis=1, inplace=True)

# Facciamo il join delle due tabelle di dati, in quanto le caratteristiche dei pokemon serviranno per allenare il modello

In [7]:
joined = all_data.join(pokemons.set_index("#"), on="First_pokemon", rsuffix="_first")
joined = joined.join(pokemons.set_index("#"), on = "Second_pokemon", rsuffix="_second")

In [ ]:
joined.head()

In [ ]:
joined.info()

# Per utilizzare un classificatore binario, reinterpreto la colonna Winner in questo moodo:
## 1 -> Ha vinto il primo pokemon
## 0 -> Ha vinto il secondo pokemon

In [10]:
def binary_winner(winner, first_pokemon, second_pokemon):
  return 1 if (first_pokemon == winner) else 0

In [ ]:
joined["Winner"] = joined.apply(lambda x: binary_winner(x['Winner'], x['First_pokemon'], x['Second_pokemon']), axis=1)
joined.head()

In [12]:
joined.drop(labels=["First_pokemon", "Second_pokemon"], axis=1, inplace=True)

# ANALISI DEI VALORI NULLI

In [ ]:
joined.info()

In [ ]:
pokemons.info()

Poichè contiene molti valori nulli, decido di droppare la colonna con il Second Type

In [15]:
joined.drop(labels=["Type 2", "Type 2_second"], axis=1, inplace=True)


# Encoding

In [16]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
joined["Legendary"] = encoder.fit_transform(joined["Legendary"])
joined["Legendary_second"] = encoder.fit_transform(joined["Legendary_second"])
joined["Type 1"] = encoder.fit_transform(joined["Type 1"])
joined["Type 1_second"] = encoder.fit_transform(joined["Type 1_second"])

In [ ]:
joined.head()

# Counting per features booleane

In [ ]:
sns.countplot(x="Legendary", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Legendary_second", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Generation", hue="Winner", data=joined)
plt.show()
sns.countplot(x="Generation_second", hue="Winner", data=joined)
plt.show()

Dai grafici riportati notiamo che la distribuzione dell'attributo di Genertion è abbastanza uniforme, perciò avrà probabilmente senso droppare questa feature. La conferma la avremo dalla heatmap.

Per valutare meglio l'influenza della feature "Legendary" invece, combiniamo i due grafici visti sopra introducento una "Legendary_diff" che varrà 1 se il prime pokemon è Leggendario, -1 se lo è il secondo, e 0 se entrambi appartengono alla stessa categoria.

In [ ]:
joined["Legendary_diff"] = joined.Legendary - joined.Legendary_second
sns.countplot(x="Legendary_diff", hue="Winner", data=joined)
plt.show()

Per i valori di Legendary invece possiamo notare come quando i due pokemon appartengono alla stessa classe (Legendary_diff = 0) allora non c'è troppa differenza tra la percentuale di vittorie o sconfitte. Mentre quando uno dei due è leggendario e l'altro no, la battaglia volge a favore del leggendario

# Analisi delle features di differenza sulle statistiche

Al fine di confrontare meglio tra loro i due pokemon che partecipano alla sfida consideriamo i valori di differenza tra le statistiche dei due, e grafichiamo la relazione tra questi valori e il valore di Winner.

In [20]:
joined["Atk_diff"] = joined.Attack - joined.Attack_second
joined["Def_diff"] = joined.Defense - joined.Defense_second
joined["Spd_diff"] = joined.Speed - joined.Speed_second
joined["Atk_sp_diff"] = joined["Sp. Atk"] - joined["Sp. Atk_second"]
joined["Def_sp_diff"] = joined["Sp. Def"] - joined["Sp. Def_second"]
joined["Hp_diff"] = joined.HP - joined.HP_second

In [ ]:
sns.displot(data = joined, x = "Spd_diff", hue="Winner", kind="kde")
plt.show()

sns.displot(data = joined, x = "Atk_diff", hue="Winner", kind="kde")
plt.show()
sns.displot(data = joined, x = "Def_diff", hue="Winner", kind="kde")
plt.show()
sns.displot(data = joined, x = "Atk_sp_diff", hue="Winner", kind="kde")
plt.show()
sns.displot(data = joined, x = "Def_sp_diff", hue="Winner", kind="kde")
plt.show()
sns.displot(data = joined, x = "Hp_diff", hue="Winner", kind="kde")
plt.show()


Dall'analisi di questi grafici si evince che la statistica più importante (fondamentale) per discernere chi sarà il vincitore è la statistica di "Speed". Qualche aiuto può venire anche da "Attack", e molto poco da "Defense".

In [ ]:
sns.displot(data = joined, x = "Defense", hue="Winner", kind="kde")
plt.show()

Difatti come si nota in questo grafico la densità di vittorie e sconfitte in relazione ai valori di Defense è abbastanza omogenea.
Con l'idea di utilizzare dei DecisionTreeClassifier potrà essere utile eliminare la feature Defense e le feature ad essa legate.

In [ ]:
sns.displot(data = joined, x = "HP", hue="Winner", kind="kde")
plt.show()

Discorso simile vale per gli HP, dove anche per grandi differenze di HP si può notare un andamento omogeneo delle vittorie all'interno del dataset.

# ANALISI DELLE CORRELAZIONI



Consideriamo anche la correlazione tra il valore di output e le differenze dei valori delle statistiche

In [ ]:
plt.figure(figsize=(25,10))
sns.heatmap(joined.corr(), annot=True, linewidths=2)

# DROP DELLE FEATURES

Data la bassa correlazione con "Type 1", "Generation", "Generation_second" decido di dropapre queste tre colonne.

Le correlazioni qui espresse sono coerenti con quanto già si poteva notare dai grafici countplot: la statistica di Generation è poco correlata, mentre Legendary ha una leggera correlazione in più e può concorrere alla classificazione.

In [25]:
joined.drop(labels=["Type 1", "Generation", "Generation_second"], axis=1, inplace=True)

Dato che la Legendary_diff ha una correlazione simile a quella di Legendary e Legendary_second elimino anche queste due feature condensando l'informazione solo sul fatto che la classe sia diversa o no.

In [26]:
joined.drop(labels=["Legendary", "Legendary_second"], axis=1, inplace=True)

Scelgo di lasciare anche le features legate alle statistiche di Defense e HP, per alleggerire la computazione e permettere una miglior generalizzazione.

In [27]:
joined.drop(labels=["Defense", "Defense_second", "Def_diff", "HP", "HP_second", "Hp_diff"], axis=1, inplace=True)

# Training

In [ ]:
joined.head()

In [29]:
train_set = joined[joined.train == 1].drop("train", axis=1)
test_set = joined[joined.train == 0].drop("train", axis=1)

y_train = train_set.Winner
x_train = train_set.drop("Winner", axis=1)
y_test = test_set.Winner
x_test = test_set.drop("Winner", axis=1)

L'uso di un RandomForestClassifier è stato preferito a quello del DecisionTree semplice in quanto l'utilizzo di più alberi di decisione diversi, e la successiva predizione basata su "votazione" effettuata dal RandomForest può aumentare l'accuracy del modello e la generalizzazione dei dati.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import plot_confusion_matrix, accuracy_score

model = RandomForestClassifier()
model = model.fit(x_train, y_train)
    
y_pred = model.predict(x_test)
accuracy = round(accuracy_score(y_pred, y_test) * 100, 2)
print("Accuracy: ", accuracy)
  
plot_confusion_matrix(model, x_test, y_test)
plt.show()